In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.datasets import load_digits
from torch.utils.data import DataLoader, Dataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading the Dataset

- In this part we load the digits dataset that we are going to use.

- We seperate the data into 3 different categories:- training, validation, and testing.

In [ ]:
train_input = np.array(pd.read_csv("/kaggle/input/digit-recognizer/train.csv"))[0:33000]

validation_input = np.array(pd.read_csv("/kaggle/input/digit-recognizer/train.csv"))[33000:]

test_input = np.array(pd.read_csv("/kaggle/input/digit-recognizer/test.csv"))

In [ ]:
train_input.shape

In [ ]:
validation_input.shape

In [ ]:
test_input.shape

In [ ]:
# This is a class for the dataset of small (8px x 8px) digits.
# Please try to understand in details how it works!
class Digits(Dataset):
    """Scikit-Learn Digits dataset."""

    def __init__(self, mode="train", transforms=None):
        self.data = []
        self.targets = []
        if mode == "train":
            for i in range(train_input.shape[0]):
                self.data.append(np.array(train_input[i][1:]).astype(np.float32))
                self.targets.append(np.array(train_input[i][0]).astype(np.float32))
            self.data = torch.tensor(np.array(self.data), requires_grad=True)
            self.targets = torch.tensor(np.array(self.targets), requires_grad=True)
                
        elif mode == "val":
            for i in range(validation_input.shape[0]):
                self.data.append(np.array(validation_input[i][1:]).astype(np.float32))
                self.targets.append(np.array(validation_input[i][0]).astype(np.float32))
            self.data = torch.tensor(np.array(self.data), requires_grad=True)
            self.targets = torch.tensor(np.array(self.targets), requires_grad=True)
            
        else:
            for i in range(test_input.shape[0]):
                self.data.append(np.array(test_input[i]).astype(np.float32))
                self.targets.append(np.array(test_input[i][0]).astype(np.float32))
            self.data = torch.tensor(np.array(self.data), requires_grad=True)
            self.targets = torch.tensor(np.array(self.targets), requires_grad=True)
            

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample_x = self.data[idx]
        sample_y = self.targets[idx]
        if self.transforms:
            sample_x = self.transforms(sample_x)
        return (sample_x, sample_y)

In [ ]:
# Here, we plot some images (8px x 8px).
digits = load_digits()
x = digits.data[:16].astype(np.float32)

fig_data, axs = plt.subplots(4, 4, figsize=(4, 4))
fig_data.tight_layout()

for i in range(4):
    for j in range(4):
        img = np.reshape(x[4 * i + j], (8, 8))
        axs[i, j].imshow(img, cmap="gray")
        axs[i, j].axis("off")

# Neural Network

- We will use Convolutional Neural Network architecture (CNN) to build our Neural Network. 


In [ ]:

class ConvolutionalNN(nn.Module):
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        
        # The Loss function 
        self.nll = nn.NLLLoss()
        
        # The Neural Network Structure
        self.Conv1 = nn.Conv2d(1,10,3)
        self.Conv2 = nn.Conv2d(10,20,5)
        
        self.BatchNorm1 = nn.BatchNorm2d(10)
        self.BatchNorm2 = nn.BatchNorm2d(20)

        self.flatten_dim = 20 * 4 * 4  # This will be calculated dynamically based on input size
        
        self.Linear1 = nn.Linear(320, 1060)
        self.Linear2 = nn.Linear(1060,500)
        self.Linear3 = nn.Linear(500,100)
        self.Linear4 = nn.Linear(100,50)
        self.Linear5 = nn.Linear(50,10)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    
    def forward(self,x):
        
        # The full forward pass of the Neural Network
            
        data = x.view(x.shape[0],1,28,28)
        self.Layer1 = F.relu(self.BatchNorm1(self.Conv1(data)))
        self.Layer2 = self.pool(self.Layer1)
        self.Layer0 = self.BatchNorm2(self.Conv2(self.Layer2))
        self.Layer3 = F.relu(self.Layer0)
        self.Layer4 = self.pool(self.Layer3)
        self.Layer4 = self.Layer4.view(self.Layer4.size(0), -1)
        self.Layer5 = self.Linear1(self.Layer4)
        self.Layer6 = F.leaky_relu(self.Layer5)
        self.Layer7 = F.leaky_relu(self.Linear2(self.Layer6))
        self.Layer8 = F.relu(self.Linear3(self.Layer7))
        self.Layer9 = F.relu(self.Linear4(self.Layer8))
        self.Layer10 = F.sigmoid(self.Layer9)
        self.Layer11 = self.Linear5(self.Layer10)
        output = F.log_softmax(self.Layer11, dim=1)
        
        return output
        
        
    def classify(self,x):
        
        classified = torch.max(x, 1)
        
        return classified 
    def loss(self,x,y):

        loss = self.nll(x, y.long())
        return loss.mean()


# Training Procedure

- Loading the training, validation, and training data

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


train = DataLoader(Digits(mode= "train"), batch_size= 64, shuffle = True)
validation = DataLoader(Digits(mode="val"), batch_size=64, shuffle = True)
test =  DataLoader(Digits(mode="test"), batch_size=28000, shuffle=False)

In [ ]:
import torch.optim as optim

num_epochs = 10
loss_validation_list = []
precision_validation = []
recall_validation = []
model = ConvolutionalNN()
try:
    model.load_state_dict(torch.load('/kaggle/working/model.pth'))
    print("Model loaded from model.pth")
except FileNotFoundError:
    print("No saved model found, training from scratch")
# We have to initialize the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum= 0.95)


In [ ]:
loss_training = 0.0
loss_training_list = []

for epoch in range(num_epochs):
    loss_training = 0.0
    model.train()
    for number, (features, targets) in enumerate(train):
        optimizer.zero_grad()
        forward = model.forward(features)
        output = model.loss(forward,targets)
        output.backward()
        optimizer.step()

        loss_training += output

        
        

    loss_training = loss_training/len(train)
    loss_training_list.append(loss_training)
    print(f"The training loss for the epoch {epoch+1} : {loss_training}")
    
    print(f"The validation statistics for epoch {epoch+1}: ")
    validation_loss = evaluate(validation,model,average="macro")
    loss_validation_list.append(validation_loss["loss"])
    precision_validation.append(validation_loss["precision"])
    recall_validation.append(validation_loss["recall"])
    print(f"The validation loss for the epoch {epoch+1} : {validation_loss}")

torch.save(model.state_dict(), "model.pth")


In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
training_list = [loss.detach().item() for loss in loss_training_list ]
x= [i for i in range(1,11)]
plt.plot(x,training_list, label= "Training")
plt.plot(x,loss_validation_list, label= "Validation")
plt.title("Validation loss vs Training loss")
plt.xlabel("Number of Epochs")
plt.ylabel("NLL loss")
plt.legend()
plt.show()


# Function to evaluate the model

- This will be used in the training loop to evaluate the model on the validation data.

- Will also be used in the end to evaluate the model on the test data.


In [ ]:
from torchmetrics import Precision, Recall

In [ ]:
def evaluate(data, model, average="None"):
    
    loss = 0.0
    precision_metric = Precision(task="multiclass", num_classes=10, average=average)
    recall_metric = Recall(task="multiclass", num_classes=10, average=average)
    
    
    model.eval()
    
    with torch.no_grad():
        
        for idx_batch, (batch,targets) in enumerate(data):
            forward = model.forward(batch)
            loss += model.loss(forward,targets)
            precision_metric(forward,targets)
            recall_metric(forward,targets)
            
            
    loss = loss/len(data)
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    
    print(f"The loss is: {loss}")
    print(f"The precision is {precision}")
    print(f"The recall is {recall}")
    
    
    return {"loss":loss, "precision": precision, "recall":recall}

# Submission

In [ ]:
for features,_ in test:
    prediction = model.forward(features)
    test_predictions = model.classify(prediction)[1]
    

In [ ]:
test_predictions[0:1].item()

In [ ]:
predictions = np.array([pred.item() for pred in test_predictions])

predictions[0:5]

In [ ]:
ImageIds = np.arange(1,28001)

In [ ]:
Dataframe = {
    "ImageId": ImageIds,
    "Label": predictions
}

Df = pd.DataFrame(Dataframe)

Df.set_index("ImageId", inplace=True)

In [ ]:
Df.head()

In [ ]:
Df.to_csv("Submission.csv")